#### Relevant Imports

In [1]:
from sqlalchemy import create_engine, text, inspect
import json
from collections import defaultdict
import ast
from groq import Groq
import dotenv
import os

**SQLite DB**  
Let us connect to the SQLite Sample Database what we have  


In [2]:
# There is an engine instance created, which can handle multiple connetions
DB_File = "Sample_1 - Copy.db"

if os.path.exists (DB_File):
    sql_engine = create_engine("sqlite:///"+DB_File)
    conn_1 = sql_engine.connect ()
else:
    print ("DB Files does not exist")

In [ ]:
# Load Env variable and define a Groq Client
dotenv.load_dotenv ()
client_gr = Groq()
# client_gr = Groq (api_key='Your Key')
model_gr = "llama-3.3-70b-versatile"

**Get the Schema**  
Function to fetch the details of all tables present in the DB  
Also fetch the table schema for all the tables present

In [ ]:
S_Instr = "You are given a SQL table name and the schema that provides key details of columns.\
          Understand each column. Understand how columns fit in the overall table\
          Don't make any assumption or cookup data\
          You need to provide a summary description about what this table contains\
          Provide summary in max 3 sentences. No additional text"

def get_schema_from_engine(engine):
    """
    Extracts all table names and compact table schema descriptions 
    from a SQLAlchemy engine connection.

    Returns:
        tuple: (table_names, schema_dict)
    """
    inspector = inspect(engine)
    table_names = inspector.get_table_names()

    schema_dict = {}
    tables_dict = {}

    for table in table_names:
        columns = inspector.get_columns(table)
        pk_columns = inspector.get_pk_constraint(table).get("constrained_columns", [])
        
        # Format the colum to make it compact
        formatted_columns = []
        for col in columns:
            col_name = col["name"]
            col_type = str(col["type"]).upper()
            is_pk = " ,PK" if col_name in pk_columns else ""
            formatted_columns.append(f"{col_name} ({col_type}{is_pk})")

        # Add to the dict 
        schema_dict[table] = formatted_columns

        # Create summary of table from the schema
        messages=[
            {
                "role": "system",
                "content": S_Instr
            },

            {
                "role": "user",
                "content": "Table Name : "+table+"\n Schema :\n"+str(formatted_columns)
            }
        ]
        completion = client_gr.chat.completions.create(
            messages=messages,
            model=model_gr,
            # temperature=0.0
        )

        tables_dict[table] = completion.choices[0].message.content

        # print (completion.choices[0].message.content)

    return tables_dict, schema_dict


> Get the Table Details and the Schema for all in a compact format

In [ ]:
tables, schema = get_schema_from_engine (sql_engine)

with open ('Schema.txt', mode='w', encoding='utf-8') as f:

    for table, desc in tables.items ():
        print (table, file=f)
        print (desc, file=f)

    for table, sch in schema.items ():
        print (table, file=f)
        print (sch, file=f)


**Get Query**  
Given User Prompt first identify the necessary SQL tables by providing all the details  
Then for the query, with identified tables and schema, get LLM to write SQL

In [ ]:
E_Instr = "You are given details of a SQL database, where each table name and its description is given\
           Also you are given a user question / statement\
           You need to identify which are the tables would be required to respond to the user query\
           If you don't see anything relevant, respond empty list\
           Respond tables names as list of strings. NO additional text"


Q_Instr = "You are given a user query and details of relevant SQL tables & Schema\
        Write SQL query to fetch the relevant details to answer the question\
        Respond only the SQL query. No additional text, no fence"


Prompt = "How many customers from USA have made total purchase of over $50?"
# Prompt = "Which are the albums have more than 5 tracks?"
# Prompt = "Are there any album that has tracks from various Genre?"
# Prompt = "Customers from India listen to which Genre of traks mainly?"
# Prompt = "Who is the support rep who has all his client from USA and made a purchase in the last month?"

messages=[
    {
        "role": "system",
        "content": E_Instr
    },

    {
        "role": "user",
        "content": "SQL DB Detail :\n"+str(tables)+"\n User Query : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,    
)

print (completion.choices[0].message.content)

# List of tables that are relevant
table_list = ast.literal_eval (completion.choices[0].message.content)

# Pull out the relevant table details and schema
tab_details = {t: tables[t] for t in table_list}
sch_details = {t : schema[t] for t in table_list}

messages=[
    {
        "role": "system",
        "content": Q_Instr
    },

    {
        "role": "user",
        "content": "Tables :\n"+str(tab_details)+"\nSchema :\n"+str(sch_details)+"\n User Query : \n"+Prompt
    }
]
completion = client_gr.chat.completions.create(
    messages=messages,
    model=model_gr,    
)

print (completion.choices[0].message.content)